## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Project paths
PROJECT_ROOT = Path('/workspaces/Recommendation-system')
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
REPORTS_FIGURES = PROJECT_ROOT / 'reports' / 'figures'

# Create directories if needed
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
REPORTS_FIGURES.mkdir(parents=True, exist_ok=True)

print("Setup complete!")

## 2. Load Raw Datasets

In [ ]:
# Load all datasets
yield_raw = pd.read_csv(DATA_RAW / 'yield.csv')
yield_df_raw = pd.read_csv(DATA_RAW / 'yield_df.csv', index_col=0)
pesticides_raw = pd.read_csv(DATA_RAW / 'pesticides.csv')
rainfall_raw = pd.read_csv(DATA_RAW / 'rainfall.csv')
temp_raw = pd.read_csv(DATA_RAW / 'temp.csv')

print("Datasets loaded successfully!")
print(f"\nDataset shapes:")
print(f"  yield_raw: {yield_raw.shape}")
print(f"  yield_df_raw: {yield_df_raw.shape}")
print(f"  pesticides_raw: {pesticides_raw.shape}")
print(f"  rainfall_raw: {rainfall_raw.shape}")
print(f"  temp_raw: {temp_raw.shape}")

## 3. Explore Individual Datasets

### 3.1 Yield Data (FAO Format)

In [ ]:
print("=" * 60)
print("YIELD DATA (FAO FORMAT)")
print("=" * 60)

print("\nColumns:")
print(yield_raw.columns.tolist())

print("\nData types:")
print(yield_raw.dtypes)

print("\nFirst 5 rows:")
yield_raw.head()

In [ ]:
print("\nBasic Statistics:")
print(f"  Number of countries: {yield_raw['Area'].nunique()}")
print(f"  Number of crops: {yield_raw['Item'].nunique()}")
print(f"  Year range: {yield_raw['Year'].min()} - {yield_raw['Year'].max()}")

print("\nCrops in FAO yield data:")
print(yield_raw['Item'].unique())

### 3.2 Pre-Merged Yield DataFrame (Primary Dataset)

In [ ]:
print("=" * 60)
print("YIELD_DF (PRE-MERGED DATASET)")
print("=" * 60)

print("\nColumns:")
print(yield_df_raw.columns.tolist())

print("\nData types:")
print(yield_df_raw.dtypes)

print("\nFirst 10 rows:")
yield_df_raw.head(10)

In [ ]:
print("\nBasic Statistics:")
print(f"  Total rows: {len(yield_df_raw)}")
print(f"  Number of countries: {yield_df_raw['Area'].nunique()}")
print(f"  Number of crops: {yield_df_raw['Item'].nunique()}")
print(f"  Year range: {yield_df_raw['Year'].min()} - {yield_df_raw['Year'].max()}")

print("\nCrops in yield_df:")
print(yield_df_raw['Item'].unique())

print("\nMissing values:")
print(yield_df_raw.isnull().sum())

In [ ]:
# Detailed statistics for numeric columns
print("\nNumeric Column Statistics:")
yield_df_raw.describe()

### 3.3 Pesticides Data

In [ ]:
print("=" * 60)
print("PESTICIDES DATA")
print("=" * 60)

print("\nColumns:")
print(pesticides_raw.columns.tolist())

print("\nFirst 5 rows:")
print(pesticides_raw.head())

print(f"\nNumber of countries: {pesticides_raw['Area'].nunique()}")
print(f"Year range: {pesticides_raw['Year'].min()} - {pesticides_raw['Year'].max()}")

### 3.4 Rainfall Data

In [ ]:
print("=" * 60)
print("RAINFALL DATA")
print("=" * 60)

# Clean column names (there's a space issue)
rainfall_raw.columns = rainfall_raw.columns.str.strip()

print("\nColumns:")
print(rainfall_raw.columns.tolist())

print("\nFirst 5 rows:")
print(rainfall_raw.head())

print(f"\nNumber of countries: {rainfall_raw['Area'].nunique()}")
print(f"Year range: {rainfall_raw['Year'].min()} - {rainfall_raw['Year'].max()}")

### 3.5 Temperature Data

In [ ]:
print("=" * 60)
print("TEMPERATURE DATA")
print("=" * 60)

print("\nColumns:")
print(temp_raw.columns.tolist())

print("\nFirst 10 rows:")
print(temp_raw.head(10))

print(f"\nNumber of countries: {temp_raw['country'].nunique()}")
print(f"Year range: {temp_raw['year'].min()} - {temp_raw['year'].max()}")

print("\nMissing values in temperature:")
print(f"  Total rows: {len(temp_raw)}")
print(f"  Missing avg_temp: {temp_raw['avg_temp'].isnull().sum()} ({100*temp_raw['avg_temp'].isnull().sum()/len(temp_raw):.2f}%)")

## 4. Data Visualization (EDA)

In [ ]:
# Use yield_df_raw as our primary dataset (already merged)
df = yield_df_raw.copy()

# Rename columns for clarity
df = df.rename(columns={
    'Area': 'country',
    'Item': 'crop',
    'Year': 'year',
    'hg/ha_yield': 'yield',
    'average_rain_fall_mm_per_year': 'rainfall_mm',
    'pesticides_tonnes': 'pesticides_tonnes',
    'avg_temp': 'avg_temp'
})

print("Working dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

### 4.1 Distribution of Target Variable (Yield)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of yield
axes[0].hist(df['yield'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Yield (hg/ha)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Crop Yield')
axes[0].axvline(df['yield'].median(), color='red', linestyle='--', label=f'Median: {df["yield"].median():,.0f}')
axes[0].legend()

# Log-transformed yield
axes[1].hist(np.log1p(df['yield']), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1].set_xlabel('Log(Yield + 1)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Log-Transformed Yield')

plt.tight_layout()
plt.savefig(REPORTS_FIGURES / 'yield_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

### 4.2 Yield by Crop Type

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Mean yield by crop
crop_yield = df.groupby('crop')['yield'].mean().sort_values(ascending=True)
crop_yield.plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_xlabel('Mean Yield (hg/ha)')
axes[0].set_title('Average Yield by Crop')

# Box plot of yield by crop
df.boxplot(column='yield', by='crop', ax=axes[1], vert=False)
axes[1].set_xlabel('Yield (hg/ha)')
axes[1].set_title('Yield Distribution by Crop')
plt.suptitle('')

plt.tight_layout()
plt.savefig(REPORTS_FIGURES / 'yield_by_crop.png', dpi=150, bbox_inches='tight')
plt.show()

### 4.3 Numeric Features Distribution

In [ ]:
numeric_cols = ['yield', 'rainfall_mm', 'pesticides_tonnes', 'avg_temp']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, col in enumerate(numeric_cols):
    df[col].hist(bins=40, ax=axes[idx], edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribution of {col}')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    
    # Add statistics
    mean_val = df[col].mean()
    median_val = df[col].median()
    axes[idx].axvline(mean_val, color='red', linestyle='-', alpha=0.8, label=f'Mean: {mean_val:.2f}')
    axes[idx].axvline(median_val, color='green', linestyle='--', alpha=0.8, label=f'Median: {median_val:.2f}')
    axes[idx].legend(fontsize=8)

plt.tight_layout()
plt.savefig(REPORTS_FIGURES / 'numeric_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

### 4.4 Correlation Analysis

In [ ]:
# Correlation matrix
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, 
            square=True, linewidths=0.5, fmt='.3f')
plt.title('Correlation Matrix of Numeric Features')
plt.tight_layout()
plt.savefig(REPORTS_FIGURES / 'correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nCorrelation with Yield:")
print(corr_matrix['yield'].sort_values(ascending=False))

### 4.5 Yield Trends Over Time

In [ ]:
# Average yield over years by crop
yearly_crop_yield = df.groupby(['year', 'crop'])['yield'].mean().unstack()

plt.figure(figsize=(14, 8))
for crop in yearly_crop_yield.columns:
    plt.plot(yearly_crop_yield.index, yearly_crop_yield[crop], label=crop, linewidth=2)

plt.xlabel('Year')
plt.ylabel('Average Yield (hg/ha)')
plt.title('Crop Yield Trends Over Time')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(REPORTS_FIGURES / 'yield_trends.png', dpi=150, bbox_inches='tight')
plt.show()

### 4.6 Scatter Plots: Features vs Yield

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

features = ['rainfall_mm', 'pesticides_tonnes', 'avg_temp']
colors = ['blue', 'green', 'red']

for idx, (feat, color) in enumerate(zip(features, colors)):
    # Sample data for better visualization
    sample = df.sample(min(5000, len(df)), random_state=42)
    axes[idx].scatter(sample[feat], sample['yield'], alpha=0.3, c=color, s=10)
    axes[idx].set_xlabel(feat)
    axes[idx].set_ylabel('Yield (hg/ha)')
    axes[idx].set_title(f'Yield vs {feat}')
    
    # Add trend line
    z = np.polyfit(sample[feat].dropna(), sample.loc[sample[feat].notna(), 'yield'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(sample[feat].min(), sample[feat].max(), 100)
    axes[idx].plot(x_line, p(x_line), 'r--', alpha=0.8, linewidth=2)

plt.tight_layout()
plt.savefig(REPORTS_FIGURES / 'scatter_features_yield.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Data Cleaning and Standardization

In [ ]:
# Check for missing values
print("Missing values before cleaning:")
print(df.isnull().sum())
print(f"\nTotal rows: {len(df)}")
print(f"Rows with any missing: {df.isnull().any(axis=1).sum()}")

In [ ]:
# Clean the dataset
df_clean = df.copy()

# 1. Strip whitespace from text columns
df_clean['country'] = df_clean['country'].str.strip()
df_clean['crop'] = df_clean['crop'].str.strip()

# 2. Ensure year is integer
df_clean['year'] = df_clean['year'].astype(int)

# 3. Handle missing values in numeric columns with median imputation
for col in ['rainfall_mm', 'pesticides_tonnes', 'avg_temp']:
    missing_count = df_clean[col].isnull().sum()
    if missing_count > 0:
        median_val = df_clean[col].median()
        df_clean[col] = df_clean[col].fillna(median_val)
        print(f"Filled {missing_count} missing values in '{col}' with median: {median_val:.2f}")

# 4. Remove rows where target is missing (should be none)
missing_yield = df_clean['yield'].isnull().sum()
if missing_yield > 0:
    df_clean = df_clean.dropna(subset=['yield'])
    print(f"Dropped {missing_yield} rows with missing yield")

# 5. Remove duplicates
duplicates = df_clean.duplicated(subset=['country', 'crop', 'year'], keep='first')
dup_count = duplicates.sum()
if dup_count > 0:
    df_clean = df_clean[~duplicates]
    print(f"Removed {dup_count} duplicate rows")

print(f"\nCleaned dataset shape: {df_clean.shape}")

In [ ]:
# Verify no missing values remain
print("Missing values after cleaning:")
print(df_clean.isnull().sum())

## 6. Dataset Validation

In [ ]:
# Validation checks
print("=" * 60)
print("DATASET VALIDATION")
print("=" * 60)

# Check 1: Required columns present
required_cols = ['country', 'crop', 'year', 'yield', 'rainfall_mm', 'pesticides_tonnes', 'avg_temp']
missing_cols = [c for c in required_cols if c not in df_clean.columns]
print(f"\n1. Required columns check: {'PASS' if not missing_cols else 'FAIL - Missing: ' + str(missing_cols)}")

# Check 2: No duplicates
dup_check = df_clean.duplicated(subset=['country', 'crop', 'year']).any()
print(f"2. No duplicates check: {'PASS' if not dup_check else 'FAIL'}")

# Check 3: Target is numeric and has no missing
yield_numeric = pd.api.types.is_numeric_dtype(df_clean['yield'])
yield_no_missing = df_clean['yield'].isnull().sum() == 0
print(f"3. Yield column valid: {'PASS' if (yield_numeric and yield_no_missing) else 'FAIL'}")

# Check 4: Feature columns present and numeric
feature_cols = ['rainfall_mm', 'pesticides_tonnes', 'avg_temp']
features_valid = all(pd.api.types.is_numeric_dtype(df_clean[c]) for c in feature_cols)
print(f"4. Feature columns valid: {'PASS' if features_valid else 'FAIL'}")

# Check 5: Reasonable value ranges
print(f"\n5. Value range checks:")
print(f"   Yield: {df_clean['yield'].min():.0f} - {df_clean['yield'].max():.0f} hg/ha")
print(f"   Rainfall: {df_clean['rainfall_mm'].min():.0f} - {df_clean['rainfall_mm'].max():.0f} mm")
print(f"   Pesticides: {df_clean['pesticides_tonnes'].min():.0f} - {df_clean['pesticides_tonnes'].max():.0f} tonnes")
print(f"   Temperature: {df_clean['avg_temp'].min():.1f} - {df_clean['avg_temp'].max():.1f} °C")

## 7. Fusion Summary

In [ ]:
# Generate comprehensive summary
print("=" * 60)
print("CONSOLIDATED DATASET SUMMARY")
print("=" * 60)

print(f"\nTotal rows: {len(df_clean):,}")
print(f"Total columns: {len(df_clean.columns)}")
print(f"\nYear range: {df_clean['year'].min()} - {df_clean['year'].max()}")
print(f"Number of unique countries: {df_clean['country'].nunique()}")
print(f"Number of unique crops: {df_clean['crop'].nunique()}")

print("\nCrops included:")
for crop in sorted(df_clean['crop'].unique()):
    count = (df_clean['crop'] == crop).sum()
    print(f"  - {crop}: {count:,} records")

print("\nTop 10 countries by record count:")
print(df_clean['country'].value_counts().head(10))

In [ ]:
# Statistical summary
print("\nStatistical Summary:")
df_clean.describe()

## 8. Save Consolidated Dataset

In [ ]:
# Save the consolidated dataset
output_path = DATA_PROCESSED / 'consolidated.csv'
df_clean.to_csv(output_path, index=False)
print(f"Consolidated dataset saved to: {output_path}")
print(f"File size: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

In [ ]:
# Save fusion summary to text file
summary_path = DATA_PROCESSED / 'fusion_summary.txt'

with open(summary_path, 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("DATA FUSION SUMMARY\n")
    f.write("=" * 60 + "\n\n")
    f.write(f"Generated: {pd.Timestamp.now()}\n\n")
    
    f.write(f"Total rows: {len(df_clean):,}\n")
    f.write(f"Total columns: {len(df_clean.columns)}\n")
    f.write(f"Columns: {', '.join(df_clean.columns)}\n\n")
    
    f.write(f"Year range: {df_clean['year'].min()} - {df_clean['year'].max()}\n")
    f.write(f"Unique countries: {df_clean['country'].nunique()}\n")
    f.write(f"Unique crops: {df_clean['crop'].nunique()}\n\n")
    
    f.write("Crops included:\n")
    for crop in sorted(df_clean['crop'].unique()):
        count = (df_clean['crop'] == crop).sum()
        f.write(f"  - {crop}: {count:,} records\n")
    
    f.write("\n" + "=" * 60 + "\n")
    f.write("FUSION STRATEGY\n")
    f.write("=" * 60 + "\n")
    f.write("- Primary data source: yield_df.csv (pre-merged dataset)\n")
    f.write("- Join keys: (country, year)\n")
    f.write("- Crop is retained from yield dataset\n")
    f.write("- Context features: rainfall_mm, pesticides_tonnes, avg_temp\n")
    f.write("- Missing values: imputed with column median\n")
    f.write("- Duplicates: removed (kept first occurrence)\n")

print(f"Fusion summary saved to: {summary_path}")

## 9. Final Verification

In [ ]:
# Reload and verify
df_verify = pd.read_csv(DATA_PROCESSED / 'consolidated.csv')
print("Verification of saved dataset:")
print(f"  Shape: {df_verify.shape}")
print(f"  Columns: {df_verify.columns.tolist()}")
print(f"  Missing values: {df_verify.isnull().sum().sum()}")
print("\n✅ Data fusion complete! Ready for modeling.")